In [1]:
from itertools import product

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from SALib import ProblemSpec

In [2]:
cost_function_results = pd.read_parquet("./output_sobol_index.parquet")
cost_function_results

station                BARENTS                            HOT                \
quantity_of_interest      mean      variance argmax      mean      variance   
0                     0.004000  3.479870e-06  205.0  0.002950  2.277094e-07   
1                     0.000044  4.303894e-10  205.0  0.000033  2.816304e-11   
2                     0.003988  3.623789e-06  208.0  0.002950  2.277094e-07   
3                     0.003999  3.456675e-06  205.0  0.002950  2.277094e-07   
4                     0.008151  5.712565e-06  254.0  0.006751  5.638046e-07   
...                        ...           ...    ...       ...           ...   
1190695               0.000499  3.177044e-07  139.0  0.000066  2.351754e-10   
1190696               0.000507  3.413467e-07  130.0  0.000067  2.065802e-10   
1190697               0.002547  1.786764e-06  216.0  0.000066  3.636120e-10   
1190698               0.001458  1.194931e-06  191.0  0.000351  7.279864e-09   
1190699               0.000485  2.876904e-07  150.0  0.000066  3.632953e-10   

station                          BATS                           PAPA  \
quantity_of_interest argmax      mean      variance argmax      mean   
0                      50.0  0.003824  2.358212e-06   97.0  0.006814   
1                      50.0  0.000043  2.916630e-10   97.0  0.000076   
2                      50.0  0.003824  2.358212e-06   97.0  0.006795   
3                      50.0  0.003824  2.358212e-06   97.0  0.006810   
4                      79.0  0.008491  4.738705e-06  111.0  0.014415   
...                     ...       ...           ...    ...       ...   
1190695               166.0  0.000074  1.607287e-09   79.0  0.000495   
1190696                48.0  0.000074  1.434071e-09   91.0  0.000498   
1190697               139.0  0.000078  1.927438e-09  109.0  0.002297   
1190698                54.0  0.000544  9.299581e-08  101.0  0.002183   
1190699               139.0  0.000074  1.603596e-09  109.0  0.000490   

station                                        GUAM                       
quantity_of_interest      variance argmax      mean      variance argmax  
0                     7.913743e-07  287.0  0.001023  6.137193e-09  113.0  
1                     9.787696e-11  287.0  0.000011  7.590464e-13  113.0  
2                     9.268995e-07  289.0  0.001023  6.137193e-09  113.0  
3                     8.046640e-07  289.0  0.001023  6.137193e-09  113.0  
4                     2.375407e-06  300.0  0.002324  1.664002e-08  113.0  
...                            ...    ...       ...           ...    ...  
1190695               3.933408e-08  112.0  0.000029  1.595918e-11  198.0  
1190696               4.364409e-08  105.0  0.000029  1.609563e-11  196.0  
1190697               4.228186e-07  155.0  0.000029  5.423783e-11  200.0  
1190698               1.692077e-07  276.0  0.000101  1.518171e-10   76.0  
1190699               3.645034e-08  121.0  0.000029  5.423783e-11  200.0  

[1197400 rows x 15 columns]

In [3]:
stations_table = pd.read_json("../1_data_processing/1_3_Sensibility/stations_locations.json")
stations_table = stations_table.set_index("name").rename(columns={"primary production": "primary_production"})
stations_table

,longitude,latitude,temperature,primary_production
name,,,,
BARENTS,26.969,74.620,4.036164,121.380569
HOT,-158.004,22.752,23.839729,254.277267
BATS,-64.200,31.604,21.537741,265.166229
PAPA,-149.996,50.006,6.785365,276.715942
GUAM,149.995,13.001,27.390701,112.102440


In [4]:
cost_function_results.describe().transpose().reset_index().groupby(["quantity_of_interest", "station"]).mean()

count          mean           std  \
quantity_of_interest station                                          
argmax               BARENTS  1197400.0  1.896542e+02  4.178356e+01   
                     BATS     1197400.0  9.754293e+01  1.639207e+01   
                     GUAM     1197400.0  1.636088e+02  7.035997e+01   
                     HOT      1197400.0  7.166184e+01  5.511521e+01   
                     PAPA     1197400.0  1.904992e+02  7.757854e+01   
mean                 BARENTS  1197400.0  5.217319e-03  5.830048e-03   
                     BATS     1197400.0  3.264357e-03  8.349648e-03   
                     GUAM     1197400.0  1.016397e-03  3.049677e-03   
                     HOT      1197400.0  2.721216e-03  7.596781e-03   
                     PAPA     1197400.0  7.486171e-03  1.019243e-02   
variance             BARENTS  1197400.0  9.218660e-06  1.197300e-05   
                     BATS     1197400.0  2.784193e-06  7.741670e-06   
                     GUAM     1197400.0  4.724727e-08  3.987894e-07   
                     HOT      1197400.0  3.620690e-07  1.520200e-06   
                     PAPA     1197400.0  2.930316e-06  6.639744e-06   

                                       min           25%           50%  \
quantity_of_interest station                                             
argmax               BARENTS  1.140000e+02  1.470000e+02  2.000000e+02   
                     BATS     1.400000e+01  9.100000e+01  9.400000e+01   
                     GUAM     2.000000e+00  1.130000e+02  1.960000e+02   
                     HOT      1.000000e+01  4.800000e+01  4.800000e+01   
                     PAPA     6.300000e+01  1.340000e+02  1.510000e+02   
mean                 BARENTS  1.319749e-08  1.092206e-03  3.089444e-03   
                     BATS     1.373881e-09  1.353290e-04  2.706947e-04   
                     GUAM     5.411974e-10  4.401971e-05  8.806513e-05   
                     HOT      1.228245e-09  1.096352e-04  2.193715e-04   
                     PAPA     1.053651e-08  1.158301e-03  3.405778e-03   
variance             BARENTS  2.086815e-16  7.815808e-07  4.312044e-06   
                     BATS     4.891095e-19  5.276497e-09  2.111376e-08   
                     GUAM     5.489673e-21  3.637505e-11  1.457946e-10   
                     HOT      7.045748e-20  6.486221e-10  2.599618e-09   
                     PAPA     3.365107e-17  1.312519e-07  8.361875e-07   

                                       75%         max  
quantity_of_interest station                            
argmax               BARENTS  2.130000e+02  322.000000  
                     BATS     1.000000e+02  365.000000  
                     GUAM     1.960000e+02  365.000000  
                     HOT      6.800000e+01  365.000000  
                     PAPA     2.870000e+02  365.000000  
mean                 BARENTS  7.166852e-03    0.036820  
                     BATS     1.688974e-03    0.089220  
                     GUAM     2.917090e-04    0.037072  
                     HOT      1.061145e-03    0.086614  
                     PAPA     9.268034e-03    0.073865  
variance             BARENTS  1.338494e-05    0.000100  
                     BATS     7.131430e-07    0.000155  
                     GUAM     9.225204e-10    0.000016  
                     HOT      5.396679e-08    0.000065  
                     PAPA     2.977691e-06    0.000114

In [5]:
quantity_of_interest = ["_".join(col).strip() for col in cost_function_results.columns.values]
quantity_of_interest

['BARENTS_mean',
 'BARENTS_variance',
 'BARENTS_argmax',
 'HOT_mean',
 'HOT_variance',
 'HOT_argmax',
 'BATS_mean',
 'BATS_variance',
 'BATS_argmax',
 'PAPA_mean',
 'PAPA_variance',
 'PAPA_argmax',
 'GUAM_mean',
 'GUAM_variance',
 'GUAM_argmax']

In [6]:
# Get unique value but keep order
stations = cost_function_results.columns.levels[0]
quantity_of_interest_function = cost_function_results.columns.levels[1]
print(stations)
print(quantity_of_interest_function)

Index(['BARENTS', 'BATS', 'GUAM', 'HOT', 'PAPA'], dtype='object', name='station')
Index(['argmax', 'mean', 'variance'], dtype='object', name='quantity_of_interest')


In [7]:
parameters_name = [
    "energy_transfert",
    "tr_0",
    "gamma_tr",
    "lambda_0",
    "gamma_lambda",
]

In [8]:
coupled_parameters_name = [
    f"({i}, {j})"
    for i, j in product(parameters_name, parameters_name)
    if parameters_name.index(i) < parameters_name.index(j)
]

In [9]:
sp = ProblemSpec(
    {
        "names": parameters_name,
        "groups": None,
        "bounds": [
            [0, 1],
            [0.1, 50],
            [-0.5, -0.0001],
            [0.1, 500],
            [-0.5, -0.0001],
        ],
        "outputs": quantity_of_interest,
    }
)

In [10]:
sp.set_results(cost_function_results.to_numpy()[: int(np.floor(np.sqrt(55000)) ** 2), ...])

Outputs:
	15 outputs: ['BARENTS_mean', 'BARENTS_variance', 'BARENTS_argmax', 'HOT_mean', 'HOT_variance', 'HOT_argmax', 'BATS_mean', 'BATS_variance', 'BATS_argmax', 'PAPA_mean', 'PAPA_variance', 'PAPA_argmax', 'GUAM_mean', 'GUAM_variance', 'GUAM_argmax']
	54756 evaluations


In [11]:
_ = sp.analyze_sobol()

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.12/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


In [12]:
all_entries = []
for st in stations:
    for qoi in quantity_of_interest_function:
        index = pd.MultiIndex.from_product([[st], [qoi], parameters_name], names=["Station", "QoI", "Parameter"])
        data = pd.DataFrame(
            np.asarray([sp.analysis[f"{st}_{qoi}"]["ST"], sp.analysis[f"{st}_{qoi}"]["ST_conf"]]).T,
            index=index,
            columns=["ST", "ST_conf"],
        )

        all_entries.append(data)
sobol_total_order = pd.concat(all_entries).reset_index()
sobol_total_order["is_hot"] = sobol_total_order["Station"].isin(
    stations_table.reset_index().query("temperature > 10")["name"]
)
sobol_total_order["is_productive"] = sobol_total_order["Station"].isin(
    stations_table.reset_index().query("primary_production > 150")["name"]
)
sobol_total_order

,Station,QoI,Parameter,ST,ST_conf,is_hot,is_productive
0,BARENTS,argmax,energy_transfert,0.410912,0.027420,False,False
1,BARENTS,argmax,tr_0,0.357162,0.019179,False,False
2,BARENTS,argmax,gamma_tr,0.034118,0.002565,False,False
3,BARENTS,argmax,lambda_0,0.876789,0.033397,False,False
4,BARENTS,argmax,gamma_lambda,0.840164,0.031822,False,False
...,...,...,...,...,...,...,...
70,PAPA,variance,energy_transfert,0.488864,0.081273,False,True
71,PAPA,variance,tr_0,0.432385,0.097837,False,True
72,PAPA,variance,gamma_tr,0.002714,0.000616,False,True
73,PAPA,variance,lambda_0,0.816263,0.126979,False,True


In [13]:
all_entries = []
for st in stations:
    for qoi in quantity_of_interest_function:
        index = pd.MultiIndex.from_product([[st], [qoi], parameters_name], names=["Station", "QoI", "Parameter"])
        data = pd.DataFrame(
            np.asarray([sp.analysis[f"{st}_{qoi}"]["S1"], sp.analysis[f"{st}_{qoi}"]["S1_conf"]]).T,
            index=index,
            columns=["S1", "S1_conf"],
        )

        all_entries.append(data)
sobol_first_order = pd.concat(all_entries).reset_index()
sobol_first_order["is_hot"] = sobol_first_order["Station"].isin(
    stations_table.reset_index().query("temperature > 10")["name"]
)
sobol_first_order["is_productive"] = sobol_first_order["Station"].isin(
    stations_table.reset_index().query("primary_production > 150")["name"]
)
sobol_first_order

,Station,QoI,Parameter,S1,S1_conf,is_hot,is_productive
0,BARENTS,argmax,energy_transfert,0.031184,0.026531,False,False
1,BARENTS,argmax,tr_0,0.071883,0.024577,False,False
2,BARENTS,argmax,gamma_tr,-0.000500,0.007219,False,False
3,BARENTS,argmax,lambda_0,0.276074,0.036098,False,False
4,BARENTS,argmax,gamma_lambda,0.241149,0.036542,False,False
...,...,...,...,...,...,...,...
70,PAPA,variance,energy_transfert,0.044374,0.018680,False,True
71,PAPA,variance,tr_0,0.028525,0.015437,False,True
72,PAPA,variance,gamma_tr,0.000508,0.001923,False,True
73,PAPA,variance,lambda_0,0.088591,0.041282,False,True


---

# Plotting the Sobol results


In [14]:
def plot_total_analysis(sobol: ProblemSpec):
    figures = []
    for qoi in sobol.analysis:
        fig = make_subplots(
            rows=1, cols=3, subplot_titles=("1st-order Sobol index", "2nd-order Sobol index", "Total Sobol index")
        )
        fig.add_trace(
            go.Bar(
                x=parameters_name,
                y=sobol.analysis[qoi]["S1"],
                error_y={"type": "data", "array": sobol.analysis[qoi]["S1_conf"]},
                name="1st-order Sobol index",
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Bar(
                x=coupled_parameters_name,
                y=list(filter(np.isfinite, sobol.analysis[qoi]["S2"].flatten())),
                error_y={"type": "data", "array": list(filter(np.isfinite, sobol.analysis[qoi]["S2_conf"].flatten()))},
                name="2nd-order Sobol index",
            ),
            row=1,
            col=2,
        )
        fig.add_trace(
            go.Bar(
                x=parameters_name,
                y=sobol.analysis[qoi]["ST"],
                error_y={"type": "data", "array": sobol.analysis[qoi]["ST_conf"]},
                name="Total Sobol index",
            ),
            row=1,
            col=3,
        )
        fig.update_layout(showlegend=False, title_text=f"Sobol indices for {qoi}")
        figures.append(fig)
    return figures

In [15]:
for fig in plot_total_analysis(sp):
    fig.show()

## Aggregated plots


### Total order Sobol indices


In [16]:
fig = px.bar(
    sobol_total_order,
    x="Station",
    facet_col="Parameter",
    y="ST",
    error_y="ST_conf",
    facet_row="QoI",
    color="is_hot",
    pattern_shape="is_productive",
    title="Total order Sobol index",
)

fig.update_layout(
    title="Total order Sobol index",
    width=1200,
    height=800,
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

In [17]:
fig.write_html("sobol_total_order.html")

### First order Sobol indices


In [18]:
fig = px.bar(
    sobol_first_order,
    x="Station",
    facet_col="Parameter",
    y="S1",
    error_y="S1_conf",
    facet_row="QoI",
    color="is_hot",
    pattern_shape="is_productive",
    title="First order Sobol index",
)

fig.update_layout(width=1200, height=800)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

In [19]:
fig.write_html("sobol_first_order.html")